# 加速：非同步爬蟲

* 了解非同步爬蟲加速原理與實作

## 作業目標

* 比較一下非同步爬蟲跟多線程爬蟲的差異是什麼？各自的優缺點為何？

Ans1: 非同步是利用程式中等待回應的時間再利用 CPU 處理額外步驟, 所以可以是單線程; 而多線程則是使用平行處理方式, 但各線程中間仍會空出等待時間

Ans2: 非同步優點在於充分利用 CPU 比較不會有閒置的等待, 但多線程平行處理則更能節省時間, 但更能提升速度的方式則是非同步加上多線程

In [1]:
import requests
from bs4 import BeautifulSoup
import random
import time
import json
import _thread

requests.adapters.DEFAULT_RETRIES = 10

import aiohttp, asyncio
import nest_asyncio
nest_asyncio.apply()


In [2]:
proxy_ips = []
response = requests.get('https://free-proxy-list.net/')
soup = BeautifulSoup(response.text, 'html.parser')

proxy_info = soup.find('tbody').find_all('tr')

for proxy in proxy_info:
    tmp = proxy.find_all('td')
    ip = tmp[0].text+':'+tmp[1].text    
    proxy_ips.append(ip)

print(len(proxy_ips))
print(proxy_ips)

300
['119.206.242.196:80', '161.202.226.194:8123', '167.71.5.83:8080', '139.59.1.14:3128', '138.68.60.8:8080', '175.141.69.200:80', '119.28.65.21:80', '46.4.129.54:80', '167.172.109.12:42792', '208.80.28.208:8080', '218.147.194.14:80', '173.212.202.65:80', '41.59.90.92:80', '125.141.117.28:80', '54.212.157.49:80', '36.65.42.59:3128', '54.92.120.94:80', '5.189.133.231:80', '69.195.157.163:8083', '103.76.50.195:80', '154.16.63.16:3128', '110.164.91.50:51065', '5.252.161.48:3128', '154.16.202.22:3128', '161.35.70.249:8080', '91.7.104.45:80', '191.96.71.118:3128', '3.85.95.1:80', '161.35.127.83:80', '69.27.207.166:80', '178.45.209.131:8080', '94.241.227.26:8080', '178.45.14.9:8080', '110.74.222.159:40348', '180.250.12.10:80', '185.74.4.47:8080', '128.199.1.186:3128', '88.198.50.103:8080', '167.172.191.249:41714', '85.25.15.217:3838', '51.15.149.176:3838', '200.41.186.100:8080', '165.22.66.91:80', '139.162.78.109:8080', '88.198.24.108:8080', '35.181.10.249:80', '175.141.69.203:80', '68.94.1

In [3]:
def verify_proxy(ip, l):
    print(ip)
    try:
        resp = requests.get('https://httpbin.org/ip', proxies={'http': ip, 'https': ip}, headers = {'Connection': 'close'})
        origin = json.loads(resp.text)['origin']
        address = ip.split(':')[0]
        if address == origin:
            l.append(ip)
    except:
        pass

In [4]:
def normal(proxy_ips, l):
    for i, ip in enumerate(proxy_ips):
        print(i, ':')
        verify_proxy(ip, l)

proxy_list1 = []

t1 = time.time()
normal(proxy_ips, proxy_list1)

0 :
119.206.242.196:80
1 :
161.202.226.194:8123
2 :
167.71.5.83:8080
3 :
139.59.1.14:3128
4 :
138.68.60.8:8080
5 :
175.141.69.200:80
6 :
119.28.65.21:80
7 :
46.4.129.54:80
8 :
167.172.109.12:42792
9 :
208.80.28.208:8080
10 :
218.147.194.14:80
11 :
173.212.202.65:80
12 :
41.59.90.92:80
13 :
125.141.117.28:80
14 :
54.212.157.49:80
15 :
36.65.42.59:3128
16 :
54.92.120.94:80
17 :
5.189.133.231:80
18 :
69.195.157.163:8083
19 :
103.76.50.195:80
20 :
154.16.63.16:3128
21 :
110.164.91.50:51065
22 :
5.252.161.48:3128
23 :
154.16.202.22:3128
24 :
161.35.70.249:8080
25 :
91.7.104.45:80
26 :
191.96.71.118:3128
27 :
3.85.95.1:80
28 :
161.35.127.83:80
29 :
69.27.207.166:80
30 :
178.45.209.131:8080
31 :
94.241.227.26:8080
32 :
178.45.14.9:8080
33 :
110.74.222.159:40348
34 :
180.250.12.10:80
35 :
185.74.4.47:8080
36 :
128.199.1.186:3128
37 :
88.198.50.103:8080
38 :
167.172.191.249:41714
39 :
85.25.15.217:3838
40 :
51.15.149.176:3838
41 :
200.41.186.100:8080
42 :
165.22.66.91:80
43 :
139.162.78.109:808

In [5]:
print(proxy_list1)
print("Normal total time:", time.time()-t1)

['161.202.226.194:8123', '85.25.15.217:3838', '60.254.50.162:3128', '54.233.120.113:8080', '110.232.86.52:53281', '185.189.211.70:8080', '69.64.48.110:3838', '116.212.129.58:59557', '36.37.177.186:8080', '202.162.198.93:8080', '103.83.116.106:55443', '212.83.189.92:3838', '51.15.157.183:3838', '212.129.27.126:3838', '118.175.93.94:45175', '37.111.42.210:8080', '38.126.254.114:58768', '128.199.253.159:8118', '103.109.58.54:8080', '51.15.157.182:3838', '201.142.225.244:8080', '118.174.233.45:44061', '69.64.49.199:3838', '195.238.71.50:53281', '110.77.242.63:8080', '95.106.210.111:8080', '45.177.108.170:999', '139.255.31.146:8080', '108.61.75.207:8080', '120.29.124.131:8080', '51.75.147.33:3128', '51.15.158.253:3838', '139.5.16.174:59933', '1.20.100.227:57396', '159.192.97.80:43278', '103.254.167.74:34327', '175.100.103.132:55667', '83.69.93.64:44331', '82.200.233.4:3128', '197.159.23.174:39150']
Normal total time: 4834.931456565857


In [7]:
def get(ip):
    resp = requests.get('https://httpbin.org/ip', proxies={'http': ip, 'https': ip}, headers = {'Connection': 'close'}, timeout=10)
    time.sleep(10)
    return resp

async def job(t, ip):
    print('Start Job ', t)
    try:
        resp = await loop.run_in_executor(None, get, ip)
        print('Done Job', t)
        return json.loads(resp.text)['origin']
    except:
        print('Job ', t, ' Fail..')
        return None

async def main(loop):
#   async with aiohttp.ClientSession() as session:                  #官網推薦建立Session的形式,也可以直接用request
    for i, ip in enumerate(proxy_ips, start=1):
        print('Connecting ', i, '. ', ip, '...')
        for t in range(10):
            tasks = [loop.create_task(job(t, ip))]
        finished, unfinished = await asyncio.wait(tasks)            #收集完成的結果,會返回完成的和沒完成的,等全部都完成了才返回
        results = [r.result() for r in finished]                #獲取所有結果
        for r in finished:
            if r.result() != None:
                all_results.append(r.result())
                break
        else:
            all_results.append(None)


# async def main(loop):
#     all_result = []
#     for ip in proxy_ips:
#         task = [loop.create_task(job(ip)) for i in range(10)]
#         finished, unfinished = await asyncio.wait(task)
#         for r in finished:
#             if r.result() != None:
#                 all_result.append(r.result())
#                 break
#         else:
#             all_result.append(None)
      
#     print(len(all_result))
#     print(all_result)

all_results = []
t1 = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
# loop.close()
print("Async total time:", time.time() - t1)

print(len(all_results))
print(all_results)

Connecting  1 .  119.206.242.196:80 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  0  Fail..
Job  1  Fail..
Job  2  Fail..
Job  3  Fail..
Job  4  Fail..
Job  6  Fail..
Job  7  Fail..
Job  8  Fail..
Job  5  Fail..
Job  9  Fail..
Connecting  2 .  161.202.226.194:8123 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Done Job 5
Done Job 1
Done Job 0
Done Job 3
Done Job 4
Done Job 2
Done Job 7
Done Job 8
Done Job 6
Done Job 9
Connecting  3 .  167.71.5.83:8080 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  2  Fail..
Job  5  Fail..
Job  1  Fail..
Job  7  Fail..
Job  3  Fail..
Job  0  Fail..
Job  9  Fail..
Connecting  4 .  139.59.1.14:3128 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job

Job  7  Fail..
Job  2  Fail..
Job  9  Fail..
Job  0  Fail..
Job  5  Fail..
Job  4  Fail..
Job  6  Fail..
Job  1  Fail..
Connecting  28 .  3.85.95.1:80 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  7  Fail..
Job  8  Fail..
Job  3  Fail..
Job  5  Fail..
Job  8  Fail..
Job  7  Fail..
Job  4  Fail..
Job  1  Fail..
Job  6  Fail..
Job  3  Fail..
Job  2  Fail..
Job  0  Fail..
Job  9  Fail..
Connecting  29 .  161.35.127.83:80 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  0  Fail..
Job  3  Fail..
Job  1  Fail..
Job  4  Fail..
Job  6  Fail..
Job  8  Fail..
Job  7  Fail..
Job  5  Fail..
Job  2  Fail..
Job  9  Fail..
Connecting  30 .  69.27.207.166:80 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  1  Fail..
Job  3  Fail..
Jo

Job  5  Fail..
Job  1  Fail..
Job  7  Fail..
Job  9  Fail..
Connecting  54 .  134.209.29.120:3128 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  2  Fail..
Job  6  Fail..
Job  4  Fail..
Job  8  Fail..
Job  0  Fail..
Job  3  Fail..
Job  5  Fail..
Job  1  Fail..
Job  4  Fail..
Job  2  Fail..
Job  6  Fail..
Job  9  Fail..
Job  0  Fail..
Connecting  55 .  159.203.61.169:3128 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  8  Fail..
Job  4  Fail..
Job  0  Fail..
Job  2  Fail..
Job  5  Fail..
Job  1  Fail..
Job  6  Fail..
Job  8  Fail..
Job  7  Fail..
Job  3  Fail..
Job  9  Fail..
Connecting  56 .  191.96.42.80:8080 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  7  Fail..
Job  1  Fail..
Job  3  Fail..
Job  0  Fail..
Job  5

Job  0  Fail..
Job  3  Fail..
Job  4  Fail..
Job  1  Fail..
Job  5  Fail..
Job  8  Fail..
Job  2  Fail..
Job  6  Fail..
Job  7  Fail..
Job  9  Fail..
Connecting  80 .  163.172.48.160:3838 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  1  Fail..
Job  3  Fail..
Job  2  Fail..
Job  5  Fail..
Job  8  Fail..
Job  6  Fail..
Job  7  Fail..
Job  9  Fail..
Connecting  81 .  14.143.168.230:8080 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Done Job 4
Done Job 0
Job  1  Fail..
Job  2  Fail..
Job  6  Fail..
Job  3  Fail..
Job  0  Fail..
Job  4  Fail..
Job  5  Fail..
Job  7  Fail..
Job  8  Fail..
Done Job 9
Connecting  82 .  192.109.165.235:80 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  9  Fail..
Job  8  Fail..
Connecting  83 . 

Job  1  Fail..
Job  4  Fail..
Job  2  Fail..
Job  0  Fail..
Job  9  Fail..
Job  6  Fail..
Job  8  Fail..
Job  3  Fail..
Job  5  Fail..
Job  7  Fail..
Connecting  106 .  180.211.183.178:60604 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  1  Fail..
Job  3  Fail..
Job  2  Fail..
Job  0  Fail..
Job  9  Fail..
Job  7  Fail..
Job  4  Fail..
Connecting  107 .  103.113.197.11:8080 ...
Job  8  Fail..
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  6  Fail..
Job  5  Fail..
Job  1  Fail..
Job  2  Fail..
Job  0  Fail..
Job  3  Fail..
Job  5  Fail..
Job  8  Fail..
Job  6  Fail..
Job  7  Fail..
Job  9  Fail..
Job  4  Fail..
Connecting  108 .  180.248.105.227:3128 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  3  Fail..
Job  0  Fail.

...

Done Job 3
Done Job 0
Done Job 5
Job  3  Fail..
Job  0  Fail..
Job  1  Fail..
Job  2  Fail..
Job  7  Fail..
Job  6  Fail..
Job  8  Fail..
Job  9  Fail..
Job  4  Fail..
Job  5  Fail..
Connecting  184 .  62.23.15.92:3128 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  4  Fail..
Job  0  Fail..
Job  1  Fail..
Job  2  Fail..
Job  3  Fail..
Job  5  Fail..
Job  8  Fail..
Job  6  Fail..
Job  9  Fail..
Job  7  Fail..
Connecting  185 .  212.83.188.82:3838 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  0  Fail..
Job  5  Fail..
Job  4  Fail..
Job  6  Fail..
Job  1  Fail..
Job  2  Fail..
Job  8  Fail..
Job  7  Fail..
Job  3  Fail..
Job  9  Fail..
Connecting  186 .  201.190.184.22:46740 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
J

Job  2  Fail..
Job  7  Fail..
Job  6  Fail..
Job  4  Fail..
Job  5  Fail..
Job  9  Fail..
Connecting  210 .  163.172.11.160:3838 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  8  Fail..
Done Job 0
Done Job 3
Done Job 1
Job  2  Fail..
Job  8  Fail..
Job  1  Fail..
Job  5  Fail..
Job  7  Fail..
Job  3  Fail..
Job  4  Fail..
Job  9  Fail..
Connecting  211 .  69.64.50.100:3838 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  6  Fail..
Job  0  Fail..
Job  0  Fail..
Job  2  Fail..
Job  3  Fail..
Job  1  Fail..
Job  5  Fail..
Job  4  Fail..
Job  7  Fail..
Job  6  Fail..
Job  8  Fail..
Job  9  Fail..
Connecting  212 .  69.64.49.199:3838 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  0  Fail..
Job  2  Fail..
Job  1  Fail..
Jo

Job  1  Fail..
Job  8  Fail..
Done Job 3
Job  2  Fail..
Job  0  Fail..
Job  1  Fail..
Job  4  Fail..
Job  8  Fail..
Job  7  Fail..
Job  3  Fail..
Job  5  Fail..
Job  6  Fail..
Done Job 9
Connecting  236 .  195.94.28.14:8080 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  1  Fail..
Job  5  Fail..
Job  3  Fail..
Job  6  Fail..
Job  4  Fail..
Job  2  Fail..
Job  0  Fail..
Job  8  Fail..
Job  7  Fail..
Job  9  Fail..
Connecting  237 .  166.155.239.80:80 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  6  Fail..
Job  5  Fail..
Job  3  Fail..
Job  2  Fail..
Job  1  Fail..
Job  9  Fail..
Job  7  Fail..
Job  8  Fail..
Job  4  Fail..
Connecting  238 .  37.221.204.206:47424 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  0  Fail

Job  0  Fail..
Job  9  Fail..
Connecting  262 .  192.109.165.16:80 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  6  Fail..
Job  2  Fail..
Job  1  Fail..
Job  5  Fail..
Job  7  Fail..
Job  3  Fail..
Job  4  Fail..
Job  8  Fail..
Job  7  Fail..
Job  4  Fail..
Job  2  Fail..
Job  8  Fail..
Job  1  Fail..
Job  6  Fail..
Job  5  Fail..
Job  0  Fail..
Job  9  Fail..
Connecting  263 .  193.183.217.22:8118 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  3  Fail..
Job  5  Fail..
Job  9  Fail..
Connecting  264 .  105.27.237.24:80 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  1  Fail..
Job  8  Fail..
Job  6  Fail..
Job  3  Fail..
Job  7  Fail..
Job  2  Fail..
Job  4  Fail..
Job  0  Fail..
Done Job 2
Done Job 0
Done Job 3
Job

Job  8  Fail..
Job  1  Fail..
Job  0  Fail..
Job  5  Fail..
Job  2  Fail..
Job  7  Fail..
Job  3  Fail..
Job  6  Fail..
Job  9  Fail..
Job  4  Fail..
Connecting  288 .  118.174.220.61:60070 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  1  Fail..
Job  0  Fail..
Job  2  Fail..
Job  4  Fail..
Job  3  Fail..
Job  5  Fail..
Job  6  Fail..
Job  7  Fail..
Job  8  Fail..
Job  9  Fail..
Connecting  289 .  41.65.174.85:8080 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  0  Fail..
Job  1  Fail..
Job  2  Fail..
Job  4  Fail..
Job  6  Fail..
Job  7  Fail..
Job  5  Fail..
Job  8  Fail..
Job  3  Fail..
Job  9  Fail..
Connecting  290 .  51.15.145.112:3838 ...
Start Job  0
Start Job  1
Start Job  2
Start Job  3
Start Job  4
Start Job  5
Start Job  6
Start Job  7
Start Job  8
Start Job  9
Job  2  Fail..
Job  1  Fail..
Job